In [4]:
from __future__ import print_function
import torch
x = torch.Tensor(5, 3)  # 构造一个未初始化的5*3的矩阵
print(f'Tensor:{x}')
x = torch.rand(5, 3)  # 构造一个随机初始化的矩阵
print(f'rand:{x})
# 此处在notebook中输出x的值来查看具体的x内容
print(f'size:{x.size()}')

#NOTE: torch.Size 事实上是一个tuple, 所以其支持相关的操作*
y = torch.rand(5, 3)
print(f'torch rand 53 y:{y}')
#此处 将两个同形矩阵相加有两种语法结构
x + y # 语法一
z = torch.add(x, y) # 语法二
print(f'torch add:{z}')

# 另外输出tensor也有两种写法
result = torch.Tensor(5, 3) # 语法一
torch.add(x, y, out=result) # 语法二
y.add_(x) # 将y与x相加
print(f'把x添加到y中:{y}')
# 特别注明：任何可以改变tensor内容的操作都会在方法名后加一个下划线'_'
# 例如：x.copy_(y), x.t_(), 这俩都会改变x的值。

#另外python中的切片操作也是资次的。
print(x)
print(x[:,1])#这一操作会输出x矩阵的第二列的所有值

SyntaxError: EOL while scanning string literal (<ipython-input-4-dd38e596e6eb>, line 6)

In [8]:

# 与 numpy 的类型转换
# 此处演示tensor和numpy数据结构的相互转换
a = torch.ones(5)
b = a.numpy()
print(f'修改前的a {a} \n {b}')
# 此处演示当修改numpy数组之后,与之相关联的tensor也会相应的被修改
a.add_(1)
print(f'add_ 1 修改后的a:{a}')
print(f'同时b也被修改:{b}')

# 将numpy的Array转换为torch的Tensor
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(f'numpy a:{a}')
print(f'torch b:{b}')

# 另外除了CharTensor之外，所有的tensor都可以在CPU运算和GPU运算之间相互转换
# 使用CUDA函数来将Tensor移动到GPU上
# 当CUDA可用时会进行GPU的运算
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y
print(f'x:{x}')
print(f'y:{y}')

修改前的a tensor([1., 1., 1., 1., 1.]) 
 [1. 1. 1. 1. 1.]
add_ 1 修改后的a:tensor([2., 2., 2., 2., 2.])
同时b也被修改:[2. 2. 2. 2. 2.]
numpy a:[2. 2. 2. 2. 2.]
torch b:tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
x:tensor([[0.3222, 0.0975, 0.2033],
        [0.1564, 0.3884, 0.3813],
        [0.9382, 0.6598, 0.4227],
        [0.5390, 0.3094, 0.3635],
        [0.0330, 0.6243, 0.3984]])
y:tensor([[1.1796, 1.0717, 0.8452],
        [1.0343, 0.9335, 1.2545],
        [1.7375, 1.5444, 1.0057],
        [1.0973, 0.3226, 0.9820],
        [0.6270, 1.0201, 0.5872]])


In [15]:
# PyTorch 中所有的神经网络都来自于autograd包
# autograd 是自动求导的意思

from torch.autograd import Variable    # PyTorch 最核心的类 Variable
x = Variable(torch.ones(2, 2), requires_grad = True)
y = x + 2
print(f'x:{x}, \n y:{y}')
# print(y.creator)

# y 是作为一个操作的结果创建的因此y有一个creator 
z = y * y * 3
out = z.mean()
print(f'z mean out:{out}')
# 现在我们来使用反向传播
print(out.backward())

# out.backward()和操作out.backward(torch.Tensor([1.0]))是等价的
# 在此处输出 d(out)/dx
print(x.grad)

x = torch.randn(3)
print(f'x:{x}')
x = Variable(x, requires_grad = True)
print(f'自动求导 x:{x}')
y = x * 2
print(f'自动求导*2 y:{y}')
while y.data.norm() < 1000:
    y = y * 2
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
print(f'gradients:{gradients}')
y.backward(gradients)
print(x.grad)


x:tensor([[1., 1.],
        [1., 1.]], requires_grad=True), 
 y:tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
z mean out:27.0
None
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
x:tensor([ 1.3697, -1.6832,  0.3149])
自动求导 x:tensor([ 1.3697, -1.6832,  0.3149], requires_grad=True)
自动求导*2 y:tensor([ 2.7394, -3.3664,  0.6297], grad_fn=<MulBackward0>)
gradients:tensor([1.0000e-01, 1.0000e+00, 1.0000e-04])
tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [16]:
# 定义一个前馈深度神经网络
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    """仅仅需要定义一个forward函数就可以了，backward会自动地生成。

    你可以在forward函数中使用所有的Tensor中的操作。

    模型中可学习的参数会由net.parameters()返回。"""
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120) # an affine operation: y = Wx + b
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # If the size is a square you can only specify a single number
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

'''神经网络的输出结果是这样的
Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)
'''

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


'神经网络的输出结果是这样的\nNet (\n  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))\n  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))\n  (fc1): Linear (400 -> 120)\n  (fc2): Linear (120 -> 84)\n  (fc3): Linear (84 -> 10)\n)\n'

In [18]:
params = list(net.parameters())
print(f'len params:{len(params)}')
print(f'params size:{params[0].size()}') # conv1's .weight

input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(f'out 1:{out}')
'''out 的输出结果如下
Variable containing:
-0.0158 -0.0682 -0.1239 -0.0136 -0.0645  0.0107 -0.0230 -0.0085  0.1172 -0.0393
[torch.FloatTensor of size 1x10]
'''

net.zero_grad() # 对所有的参数的梯度缓冲区进行归零
out.backward(torch.randn(1, 10)) # 使用随机的梯度进行反向传播
print(f'out:{out}')

len params:10
params size:torch.Size([6, 1, 5, 5])
out 1:tensor([[ 0.1054, -0.0303, -0.0447,  0.0809, -0.1444,  0.0978,  0.0502, -0.0252,
          0.0375, -0.0097]], grad_fn=<AddmmBackward>)
out:tensor([[ 0.1054, -0.0303, -0.0447,  0.0809, -0.1444,  0.0978,  0.0502, -0.0252,
          0.0375, -0.0097]], grad_fn=<AddmmBackward>)


In [19]:
output = net(input)
target = Variable(torch.range(1, 10))  # a dummy target, for example
criterion = nn.MSELoss()
loss = criterion(output, target)
"""loss 计算流程图 input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
      -> view -> linear -> relu -> linear -> relu -> linear 
      -> MSELoss
      -> loss"""
print(f'损失值:{loss}')
'''loss的值如下
Variable containing:
 38.5849
[torch.FloatTensor of size 1]
'''

损失值:38.40768814086914


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


'loss的值如下\nVariable containing:\n 38.5849\n[torch.FloatTensor of size 1]\n'

In [21]:
# 代价函数，损失函数的使用 Relu 
# For illustration, let us follow a few steps backward
'''
print(loss.creator) # MSELoss
print(loss.creator.previous_functions[0][0]) # Linear
print(loss.creator.previous_functions[0][0].previous_functions[0][0]) # ReLU
'''

'''
<torch.nn._functions.thnn.auto.MSELoss object at 0x7fe8102dd7c8>
<torch.nn._functions.linear.Linear object at 0x7fe8102dd708>
<torch.nn._functions.thnn.auto.Threshold object at 0x7fe8102dd648>
'''

# 现在我们应当调用loss.backward(), 之后来看看 conv1's在进行反馈之后的偏置梯度如何
net.zero_grad() # 归零操作
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

''' 这些步骤的输出结果如下
conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
 0.0346
-0.0141
 0.0544
-0.1224
-0.1677
 0.0908
[torch.FloatTensor of size 6]
'''

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0117,  0.0936,  0.0021, -0.0919,  0.0471,  0.0298])


' 这些步骤的输出结果如下\nconv1.bias.grad before backward\nVariable containing:\n 0\n 0\n 0\n 0\n 0\n 0\n[torch.FloatTensor of size 6]\n\nconv1.bias.grad after backward\nVariable containing:\n 0.0346\n-0.0141\n 0.0544\n-0.1224\n-0.1677\n 0.0908\n[torch.FloatTensor of size 6]\n'

In [24]:
# 权重的使用
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    
# 然而在你使用神经网络的时候你想要使用不同种类的方法诸如：SGD, Nesterov-SGD, Adam, RMSProp, etc.
import torch.optim as optim
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update
print(f'loss:{loss}, \n optimizer:{optimizer}')

loss:36.02983093261719, 
 optimizer:SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [25]:
# 数据的加载和使用
# 通常来讲，当你处理图像，声音，文本，视频时需要使用python中其他独立的包来将他们转换为numpy中的数组，之后再转换为torch.*Tensor
# 图像的话，可以用Pillow, OpenCV。
# 声音处理可以用scipy和librosa。
# 文本的处理使用原生Python或者Cython以及NLTK和SpaCy都可以。
# 图像，我们有torchvision这个包可用,其中包含了一些现成的数据集如：Imagenet, CIFAR10, MNIST等等。同时还有一些转换图像用的工具。 这非常的方便并且避免了写样板代码。

# 这里使用CIFAR10数据集。 我们要进行的分类的类别有：'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'。 这个数据集中的图像都是3通道，32x32像素的图片

In [26]:
# 实验 图片分类
# 步骤
"""使用torchvision来读取并预处理CIFAR10数据集
定义一个卷积神经网络
定义一个代价函数
在神经网络中训练训练集数据
使用测试集数据测试神经网络"""
# 1,读取数据
import torchvision
import torchvision.transforms as transforms


# torchvision数据集的输出是在[0, 1]范围内的PILImage图片。
# 我们此处使用归一化的方法将其转化为Tensor，数据范围为[-1, 1]

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, 
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, 
                                          shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(f'classes:{classes}')
'''注：这一部分需要下载部分数据集 因此速度可能会有一些慢 同时你会看到这样的输出

Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz
Extracting tar file
Done!
Files already downloaded and verified
'''

Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz


100.0%

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
classes:('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


'注：这一部分需要下载部分数据集 因此速度可能会有一些慢 同时你会看到这样的输出\n\nDownloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz\nExtracting tar file\nDone!\nFiles already downloaded and verified\n'

In [27]:
# 查看下载的图像
# functions to show an image
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
def imshow(img):
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))

# show some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s'%classes[labels[j]] for j in range(4)))

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 79, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 79, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/utils/data/_utils/col

In [28]:
ls ../

Design_Model.xline
base_code/
base_structure/
cpp_program
map_structure--wm.pdf
map_structure.xline
py_adt_stack.ipynb
py_build_structure(py实现数据结构).md
py_data_structure.md
py_data_structure.xline
py_debug_design_mode.xline
py_end_expression.py
py_scientific_computing(建模).md


In [ ]:
# 定义一个神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool  = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()


In [ ]:
# 定义代价函数和优化器
criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# 训练网络
for epoch in range(2): # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()        
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

In [ ]:
# 测试训练结果
# 通过对比神经网络给出的分类和已知的类别结果，可以得出正确与否，如果预测的正确，我们可以将样本加入正确预测的结果的列表中。
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images  查看已知分类
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s'%classes[labels[j]] for j in range(4)))

In [ ]:
# 训练的模型如何看待 已知分类
outputs = net(Variable(images))

# the outputs are energies for the 10 classes. 
# Higher the energy for a class, the more the network 
# thinks that the image is of the particular class

# So, let's get the index of the highest energy
_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s'% classes[predicted[j][0]] for j in range(4)))

'''输出结果为
Predicted:    cat plane   car plane
'''

In [ ]:
# 训练模型在整个数据集的表现
# 看上去这玩意输出的结果比随机整的要好，随机选择的话从十个中选择一个出来，准确率大概只有10%。看上去神经网络学到了点东西。
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

'''输出结果为
Accuracy of the network on the 10000 test images: 54 %
'''

In [ ]:
#  全部数据集的准确率
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

'''输出结果为
Accuracy of plane : 73 %
Accuracy of   car : 70 %
Accuracy of  bird : 52 %
Accuracy of   cat : 27 %
Accuracy of  deer : 34 %
Accuracy of   dog : 37 %
Accuracy of  frog : 62 %
Accuracy of horse : 72 %
Accuracy of  ship : 64 %
Accuracy of truck : 53 %
'''

In [ ]:
# 在GPU训练
net.cuda()

'''输出结果为
Net (
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)
'''


In [ ]:
inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())    # 每一步都需要把输入和目标传给GPU

In [ ]:
# 因为神经网络实在太小了，其中的差距并不明显 ， 没有进行CPU运算和GPU运算速度的对比